# Practice Scenario: Binary Classification of Malicious vs Benign Network Flows

Dataset: https://research.unsw.edu.au/projects/unsw-nb15-dataset

Description: The raw network packets of the **UNSW-NB 15 dataset** was created by the IXIA PerfectStorm tool in the Cyber Range Lab of UNSW Canberra for generating a hybrid of **real modern normal activities and synthetic contemporary attack behaviours**. The tcpdump tool was utilised to capture 100 GB of the raw traffic (e.g., Pcap files). This dataset has nine types of attacks, namely, `Fuzzers`, `Analysis`, `Backdoors`, `DoS`, `Exploits`, `Generic`, `Reconnaissance`, `Shellcode` and `Worms`. The Argus, Bro-IDS tools are used and twelve algorithms are developed to generate totally 49 features with the class label. These features are described in the UNSW-NB15_features.csv file.


Problem Statement: *Given 49 flow-based features, predict whether a network connection is malicious (1) or benign (0) using Logistic Regression. Achieve an acceptable AUC-ROC and interpret the top 5 drivers of malice.*

Start with the pcap files, convert them using zeek, fix the naming, create logistic regression model, scale to MLOps architecture, connect back to laptop for live tcpdump testing.

PLAN:
1. Create Logistic Regression Model with Training & Testing Data / Gather Insights
2. Move Project to MLOps environment
3. Ingestion of data should be capable in pcap formatt
4. Deploy and test model with laptop 

In [47]:
# Import Libraries
import pandas as pd 
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from statsmodels.api import OLS, add_constant
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, roc_curve, confusion_matrix
import joblib
from pathlib import Path

# Settings
pd.set_option('display.max_columns', None)
MODELS_DIR = Path('../models')
MODELS_DIR.mkdir(parents=True, exist_ok=True)

In [48]:
train = pd.read_csv('../data/UNSW_NB15_training-set.csv')
test = pd.read_csv('../data/UNSW_NB15_testing-set.csv')
display(train.head())
display(test.head())

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt,synack,ackdat,smean,dmean,trans_depth,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.121478,tcp,-,FIN,6,4,258,172,74.087490,252,254,14158.942380,8495.365234,0,0,24.295600,8.375000,30.177547,11.830604,255,621772692,2202533631,255,0.000000,0.000000,0.000000,43,43,0,0,1,0,1,1,1,1,0,0,0,1,1,0,Normal,0
1,2,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,62,252,8395.112305,503571.312500,2,17,49.915000,15.432865,61.426934,1387.778330,255,1417884146,3077387971,255,0.000000,0.000000,0.000000,52,1106,0,0,43,1,1,1,1,2,0,0,0,1,6,0,Normal,0
2,3,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,62,252,1572.271851,60929.230470,1,6,231.875571,102.737203,17179.586860,11420.926230,255,2116150707,2963114973,255,0.111897,0.061458,0.050439,46,824,0,0,7,1,2,1,1,3,0,0,0,2,6,0,Normal,0
3,4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,62,252,2740.178955,3358.622070,1,3,152.876547,90.235726,259.080172,4991.784669,255,1107119177,1047442890,255,0.000000,0.000000,0.000000,52,64,0,0,1,1,2,1,1,3,1,1,0,2,1,0,Normal,0
4,5,0.449454,tcp,-,FIN,10,6,534,268,33.373826,254,252,8561.499023,3987.059814,2,1,47.750333,75.659602,2415.837634,115.807000,255,2436137549,1977154190,255,0.128381,0.071147,0.057234,53,45,0,0,43,1,2,2,1,40,0,0,0,2,39,0,Normal,0


,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt,synack,ackdat,smean,dmean,trans_depth,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.000011,udp,-,INT,2,0,496,0,90909.0902,254,0,180363632.0,0.0,0,0,0.011,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,248,0,0,0,2,2,1,1,1,2,0,0,0,1,2,0,Normal,0
1,2,0.000008,udp,-,INT,2,0,1762,0,125000.0003,254,0,881000000.0,0.0,0,0,0.008,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,881,0,0,0,2,2,1,1,1,2,0,0,0,1,2,0,Normal,0
2,3,0.000005,udp,-,INT,2,0,1068,0,200000.0051,254,0,854400000.0,0.0,0,0,0.005,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,534,0,0,0,3,2,1,1,1,3,0,0,0,1,3,0,Normal,0
3,4,0.000006,udp,-,INT,2,0,900,0,166666.6608,254,0,600000000.0,0.0,0,0,0.006,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,450,0,0,0,3,2,2,2,1,3,0,0,0,2,3,0,Normal,0
4,5,0.000010,udp,-,INT,2,0,2126,0,100000.0025,254,0,850400000.0,0.0,0,0,0.010,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,1063,0,0,0,3,2,2,2,1,3,0,0,0,2,3,0,Normal,0


In [49]:
train['label'].value_counts(normalize=True)

label
1    0.680622
0    0.319378
Name: proportion, dtype: float64

The proportion between malicious and benign data becomes important when choosing the `class_weight` parameter to determine if it should be more balanced or if the data is already balanced.

### Check For NAN Values

In [50]:
train['service'] = train['service'].replace('-', 'unknown')
test['service'] = test['service'].replace('-', 'unknown')

In [51]:
print("Train Data")
for column in train.columns:
    if train[column].isna().sum():
        print(f"{column}: {train[column].isna().sum()}")

print("Test Data")
for column in test.columns:
    if test[column].isna().sum():
        print(f"{column}: {test[column].isna().sum()}")

Train Data
Test Data


Included another variable `split` in order to combine the data and seperate it later back into the original format

In [52]:
train['split'] = 'train'
test['split'] = 'test'

In [53]:
df = pd.concat([train, test])
df.to_csv('../data/01_raw/rawdata.csv')

### Convert Categorical to Dummy Variables
Logistic regression cannot handle dummy variables. It is also important to drop one of the dummy variables in order to avoid **multicoliniarity**

In [54]:
df_encoded = pd.get_dummies(df, columns=['proto', 'service', 'state'])
df_encoded.drop(columns=['proto_tcp', 'service_unknown', 'state_INT'], inplace=True)
df_encoded.head()

,id,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt,synack,ackdat,smean,dmean,trans_depth,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label,split,proto_3pc,proto_a/n,proto_aes-sp3-d,proto_any,proto_argus,proto_aris,proto_arp,proto_ax.25,proto_bbn-rcc,proto_bna,proto_br-sat-mon,proto_cbt,proto_cftp,proto_chaos,proto_compaq-peer,proto_cphb,proto_cpnx,proto_crtp,proto_crudp,proto_dcn,proto_ddp,proto_ddx,proto_dgp,proto_egp,proto_eigrp,proto_emcon,proto_encap,proto_etherip,proto_fc,proto_fire,proto_ggp,proto_gmtp,proto_gre,proto_hmp,proto_i-nlsp,proto_iatp,proto_ib,proto_icmp,proto_idpr,proto_idpr-cmtp,proto_idrp,proto_ifmp,proto_igmp,proto_igp,proto_il,proto_ip,proto_ipcomp,proto_ipcv,proto_ipip,proto_iplt,proto_ipnip,proto_ippc,proto_ipv6,proto_ipv6-frag,proto_ipv6-no,proto_ipv6-opts,proto_ipv6-route,proto_ipx-n-ip,proto_irtp,proto_isis,proto_iso-ip,proto_iso-tp4,proto_kryptolan,proto_l2tp,proto_larp,proto_leaf-1,proto_leaf-2,proto_merit-inp,proto_mfe-nsp,proto_mhrp,proto_micp,proto_mobile,proto_mtp,proto_mux,proto_narp,proto_netblt,proto_nsfnet-igp,proto_nvp,proto_ospf,proto_pgm,proto_pim,proto_pipe,proto_pnni,proto_pri-enc,proto_prm,proto_ptp,proto_pup,proto_pvp,proto_qnx,proto_rdp,proto_rsvp,proto_rtp,proto_rvd,proto_sat-expak,proto_sat-mon,proto_sccopmce,proto_scps,proto_sctp,proto_sdrp,proto_secure-vmtp,proto_sep,proto_skip,proto_sm,proto_smp,proto_snp,proto_sprite-rpc,proto_sps,proto_srp,proto_st2,proto_stp,proto_sun-nd,proto_swipe,proto_tcf,proto_tlsp,proto_tp++,proto_trunk-1,proto_trunk-2,proto_ttp,proto_udp,proto_unas,proto_uti,proto_vines,proto_visa,proto_vmtp,proto_vrrp,proto_wb-expak,proto_wb-mon,proto_wsn,proto_xnet,proto_xns-idp,proto_xtp,proto_zero,service_dhcp,service_dns,service_ftp,service_ftp-data,service_http,service_irc,service_pop3,service_radius,service_smtp,service_snmp,service_ssh,service_ssl,state_ACC,state_CLO,state_CON,state_ECO,state_FIN,state_PAR,state_REQ,state_RST,state_URN,state_no
0,1,0.121478,6,4,258,172,74.087490,252,254,14158.942380,8495.365234,0,0,24.295600,8.375000,30.177547,11.830604,255,621772692,2202533631,255,0.000000,0.000000,0.000000,43,43,0,0,1,0,1,1,1,1,0,0,0,1,1,0,Normal,0,train,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
1,2,0.649902,14,38,734,42014,78.473372,62,252,8395.112305,503571.312500,2,17,49.915000,15.432865,61.426934,1387.778330,255,1417884146,3077387971,255,0.000000,0.000000,0.000000,52,1106,0,0,43,1,1,1,1,2,0,0,0,1,6,0,Normal,0,train,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fal

Instead of `drop_first=True` decided to go with dropping the most occuring `proto`, `service`, and `state` to act as the baseline and avoid the perfect multicoliniarity. These values respectively include `tcp`, `unknown`, and `INT`.

Therefore, further interpretation of these categories will be with the baseline of the respective values.

In [55]:
df_clean = df_encoded.drop(columns=['id', 'attack_cat'])
df_clean.head()

,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt,synack,ackdat,smean,dmean,trans_depth,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,label,split,proto_3pc,proto_a/n,proto_aes-sp3-d,proto_any,proto_argus,proto_aris,proto_arp,proto_ax.25,proto_bbn-rcc,proto_bna,proto_br-sat-mon,proto_cbt,proto_cftp,proto_chaos,proto_compaq-peer,proto_cphb,proto_cpnx,proto_crtp,proto_crudp,proto_dcn,proto_ddp,proto_ddx,proto_dgp,proto_egp,proto_eigrp,proto_emcon,proto_encap,proto_etherip,proto_fc,proto_fire,proto_ggp,proto_gmtp,proto_gre,proto_hmp,proto_i-nlsp,proto_iatp,proto_ib,proto_icmp,proto_idpr,proto_idpr-cmtp,proto_idrp,proto_ifmp,proto_igmp,proto_igp,proto_il,proto_ip,proto_ipcomp,proto_ipcv,proto_ipip,proto_iplt,proto_ipnip,proto_ippc,proto_ipv6,proto_ipv6-frag,proto_ipv6-no,proto_ipv6-opts,proto_ipv6-route,proto_ipx-n-ip,proto_irtp,proto_isis,proto_iso-ip,proto_iso-tp4,proto_kryptolan,proto_l2tp,proto_larp,proto_leaf-1,proto_leaf-2,proto_merit-inp,proto_mfe-nsp,proto_mhrp,proto_micp,proto_mobile,proto_mtp,proto_mux,proto_narp,proto_netblt,proto_nsfnet-igp,proto_nvp,proto_ospf,proto_pgm,proto_pim,proto_pipe,proto_pnni,proto_pri-enc,proto_prm,proto_ptp,proto_pup,proto_pvp,proto_qnx,proto_rdp,proto_rsvp,proto_rtp,proto_rvd,proto_sat-expak,proto_sat-mon,proto_sccopmce,proto_scps,proto_sctp,proto_sdrp,proto_secure-vmtp,proto_sep,proto_skip,proto_sm,proto_smp,proto_snp,proto_sprite-rpc,proto_sps,proto_srp,proto_st2,proto_stp,proto_sun-nd,proto_swipe,proto_tcf,proto_tlsp,proto_tp++,proto_trunk-1,proto_trunk-2,proto_ttp,proto_udp,proto_unas,proto_uti,proto_vines,proto_visa,proto_vmtp,proto_vrrp,proto_wb-expak,proto_wb-mon,proto_wsn,proto_xnet,proto_xns-idp,proto_xtp,proto_zero,service_dhcp,service_dns,service_ftp,service_ftp-data,service_http,service_irc,service_pop3,service_radius,service_smtp,service_snmp,service_ssh,service_ssl,state_ACC,state_CLO,state_CON,state_ECO,state_FIN,state_PAR,state_REQ,state_RST,state_URN,state_no
0,0.121478,6,4,258,172,74.087490,252,254,14158.942380,8495.365234,0,0,24.295600,8.375000,30.177547,11.830604,255,621772692,2202533631,255,0.000000,0.000000,0.000000,43,43,0,0,1,0,1,1,1,1,0,0,0,1,1,0,0,train,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
1,0.649902,14,38,734,42014,78.473372,62,252,8395.112305,503571.312500,2,17,49.915000,15.432865,61.426934,1387.778330,255,1417884146,3077387971,255,0.000000,0.000000,0.000000,52,1106,0,0,43,1,1,1,1,2,0,0,0,1,6,0,0,train,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False

### Train / Test Split

Apply L1 (LASSO) if you want automatic feature selection — it will zero out uninformative dummies.

Apply L2 (ridge) if you want stability — it shrinks correlated coefficients but keeps them all in play.

In [56]:
train_df = df_clean[df_clean['split'] == 'train']
train_df.to_csv('../data/02_processed/train.csv')
test_df = df_clean[df_clean['split'] == 'test']
test_df.to_csv('../data/02_processed/test.csv')

X_train = train_df.drop(columns=['label', 'split'])
y_train = train_df['label']

X_test = test_df.drop(columns=['label', 'split'])
y_test = test_df['label']

### Feature Significance via OLS
Use OLS on the training split only (no leakage) to get approximate p-values and effect directions for each feature. This is strictly for interpretation because the classification target is binary.


In [57]:
ols_X = train_df.drop(columns=['label', 'split']).astype(float)
ols_X_const = add_constant(ols_X)
ols_model = OLS(y_train, ols_X_const).fit()

ols_summary_df = (
    pd.DataFrame(ols_model.params, columns=['coef'])
    .join(ols_model.pvalues.rename('p_value'))
    .join(ols_model.bse.rename('std_err'))
)
ols_summary_df_sorted = ols_summary_df.sort_values('p_value')

print("Top 10 statistically significant features (by p-value):")
display(ols_summary_df_sorted.head(10))

significant_mask = ols_summary_df_sorted['p_value'] < 0.05
print(f"Total features with p < 0.05: {significant_mask.sum()} out of {ols_summary_df_sorted.shape[0]}")

ols_summary_path = MODELS_DIR / 'ols_feature_significance.csv'
ols_summary_df_sorted.to_csv(ols_summary_path)
print(f"Saved full OLS summary to {ols_summary_path}")


Top 10 statistically significant features (by p-value):


,coef,p_value,std_err
dttl,0.003224,0.000000e+00,0.000023
dmean,0.000157,0.000000e+00,0.000004
service_smtp,0.230291,0.000000e+00,0.004483
service_http,0.125801,0.000000e+00,0.002730
service_dns,0.146986,0.000000e+00,0.003062
state_CON,-0.643932,0.000000e+00,0.009307
service_ftp-data,0.156328,5.256734e-297,0.004236
ct_state_ttl,0.138285,8.549008e-287,0.003814
state_REQ,-0.601329,3.131110e-285,0.016630
tcprtt,-0.274713,1.268893e-220,0.008652


Total features with p < 0.05: 178 out of 194
Saved full OLS summary to ..\models\ols_feature_significance.csv


In [58]:
print(f"Training samples: {X_train.shape[0]} | Features: {X_train.shape[1]}")
print(f"Testing samples:  {X_test.shape[0]} | Features: {X_test.shape[1]}")

Training samples: 175341 | Features: 193
Testing samples:  82332 | Features: 193


In [59]:
numeric_features = [
    'dur',
    'spkts',
    'dpkts',
    'sbytes',
    'dbytes',
    'rate',
    'sttl',
    'dttl',
    'sload',
    'dload',
    'sloss',
    'dloss',
    'sinpkt',
    'dinpkt',
    'sjit',
    'djit',
    'swin',
    'stcpb',
    'dtcpb',
    'dwin',
    'tcprtt',
    'synack',
    'ackdat',
    'smean',
    'dmean',
    'trans_depth',
    'response_body_len',
    'ct_srv_src',
    'ct_state_ttl',
    'ct_dst_ltm',
    'ct_src_dport_ltm',
    'ct_dst_sport_ltm',
    'ct_dst_src_ltm'
]

categorical_features = [col for col in X_train.columns if col not in numeric_features]

In [60]:
scaler = StandardScaler()
X_train_scaled_num = scaler.fit_transform(X_train[numeric_features])
X_test_scaled_num  = scaler.transform(X_test[numeric_features])

# Put back together: scaled numeric + untouched categorical

X_train_scaled = np.hstack([X_train_scaled_num, X_train[categorical_features].values])
X_test_scaled  = np.hstack([X_test_scaled_num, X_test[categorical_features].values])

### Train The Model

I was struggling to decide between `class_weight` as balanced, or none. Balanced I think would help as the split currently is **68% malicious** and **31% benign**. I will test both however I believe that leaving it at None will be beneficial as the majority class is malicious meaning that if left at none, the model will lean ever so slightly towards malicious which would be beneficial if high security is in the hands of the model.

In [ ]:
elastic_net = LogisticRegression(
    penalty='elasticnet',
    solver='saga',
    max_iter=1000,
    random_state=42
)

param_grid = {
    'C': [0.01, 0.1, 1.0, 10.0],
    'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]
}

total_candidates = len(param_grid['C']) * len(param_grid['l1_ratio'])
total_fits = total_candidates * 3  # cv=3
print(f"Starting Elastic Net grid search over {total_candidates} combos ({total_fits} fits)...")

grid_search = GridSearchCV(
    estimator=elastic_net,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=3,
    n_jobs=-1,
    verbose=3
)

grid_search.fit(X_train_scaled, y_train)
print("Grid search complete. Selecting best model...")
best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test_scaled)
y_pred_prob = best_model.predict_proba(X_test_scaled)[:, 1]

print(f"Best CV ROC-AUC: {grid_search.best_score_:.3f}")
print(f"Best Params: {grid_search.best_params_}")

Starting Elastic Net grid search over 20 combos (60 fits)...
Fitting 3 folds for each of 20 candidates, totalling 60 fits


c:\Users\SABIO\miniconda3\envs\py310\lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Grid search complete. Selecting best model...
Best CV ROC-AUC: 0.915
Best Params: {'C': 10.0, 'l1_ratio': 0.9}


In [37]:
grid_results = (
    pd.DataFrame(grid_search.cv_results_)
    [['param_C', 'param_l1_ratio', 'mean_test_score', 'std_test_score']]
    .sort_values('mean_test_score', ascending=False)
    .reset_index(drop=True)
)

grid_results.head()

,param_C,param_l1_ratio,mean_test_score,std_test_score
0,10.0,0.9,0.914828,0.110425
1,10.0,0.7,0.914828,0.110426
2,10.0,0.1,0.914827,0.110426
3,10.0,0.5,0.914827,0.110426
4,10.0,0.3,0.914827,0.110426


### Model Evaluation


In [61]:
print("Iterations used per class:", best_model.n_iter_)
print("Max iterations allowed:", best_model.max_iter)

accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob)
print(f'Accuracy: {accuracy:.2f}')
print(f'ROC-AUC Score: {roc_auc:.2f}')
print('\nClassification Report:')
print(classification_report(y_test, y_pred))

Iterations used per class: [100]
Max iterations allowed: 100
Accuracy: 0.80
ROC-AUC Score: 0.94

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.59      0.73     37000
           1       0.75      0.98      0.85     45332

    accuracy                           0.80     82332
   macro avg       0.85      0.79      0.79     82332
weighted avg       0.84      0.80      0.80     82332



### Additional Diagnostics
Capture confusion matrix and ROC curve coordinates for reporting/plots.


In [ ]:
cm = confusion_matrix(y_test, y_pred)
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

print("Confusion Matrix:")
print(cm)

roc_points = pd.DataFrame({'fpr': fpr, 'tpr': tpr, 'threshold': thresholds})
roc_path = MODELS_DIR / 'roc_curve_points.csv'
roc_points.to_csv(roc_path, index=False)
print(f"Saved ROC curve coordinates to {roc_path}")

Saved best model artifact to ..\models\log_reg_elasticnet.joblib


### Persist the Best Model & Artifacts
Store the tuned logistic regression, scaler, and supporting metadata so the exact pipeline can be reused for inference or reporting.


In [ ]:
artifact = {
    'model': best_model,
    'scaler': scaler,
    'numeric_features': numeric_features,
    'categorical_features': categorical_features,
    'feature_order': list(X_train.columns),
    'cv_best_score': grid_search.best_score_,
    'cv_best_params': grid_search.best_params_,
    'created_at_utc': pd.Timestamp.utcnow(),
}

artifact_path = MODELS_DIR / 'log_reg_elasticnet.joblib'
joblib.dump(artifact, artifact_path)
print(f"Saved best model artifact to {artifact_path}")

Confusion Matrix:
[[21861 15139]
 [  917 44415]]
Saved ROC curve coordinates to ..\models\roc_curve_points.csv
